In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.insert(0, "../src")

import json
import random
import numpy as np
import scipy
import sqlite3
from copy import deepcopy
from pathlib import Path
from collections import Counter, defaultdict
from joblib import Parallel, delayed

import constants
from gen.util import read_data, write_jsonl
from feverise import scifact, climatefever

In [3]:
feverp = Path("/users/k21190024/study/fact-check-transfer-learning/scratch/data/fever")
climatefdp = Path("/users/k21190024/study/fact-check-transfer-learning/scratch/dumps/feverised-climatefever")
scifactdp = Path("/users/k21190024/study/fact-check-transfer-learning/scratch/dumps/feverised-scifact")

# Generate Climate-FEVER Stratified 3-fold cross validation dataset

 - Fine-tune each fold with Climate-FEVER then validate on 1 fold Climate-FEVER and test on SciFact
 - 3-fold for computational reasons
 - Use same hyperparameter fine tuning parameters
 - This is done to reduce random effect from dataset
 
__Do I still need this?__

In [61]:
from sklearn.model_selection import StratifiedKFold

climatefd_folds_p = Path("/users/k21190024/study/fact-check-transfer-learning/scratch/dumps/feverised-climatefever/paper_all_kfold")
kfold = StratifiedKFold(n_splits=3, random_state=constants.SEED, shuffle=True)

In [62]:
cf = read_data(climatefdp / "climatefever_paper_all.jsonl")
cf_idt = np.array([[d["id"], d["label"]] for d in cf])
cf_splits = {str(i): np.take(cf, test_idx) for i, (_, test_idx) in enumerate(kfold.split(cf_idt[:, 0], cf_idt[:, 1]))}

for i, fold in cf_splits.items():
    write_jsonl(climatefd_folds_p / f"fold{i}.jsonl", list(fold))

In [67]:
from collections import Counter

check_folds = {i: Counter([doc["label"] for doc in ls]) for i, ls in cf_splits.items()}
check_folds

{'0': Counter({'SUPPORTS': 218, 'REFUTES': 85, 'NOT ENOUGH INFO': 158}),
 '1': Counter({'SUPPORTS': 218, 'REFUTES': 84, 'NOT ENOUGH INFO': 158}),
 '2': Counter({'REFUTES': 84, 'SUPPORTS': 218, 'NOT ENOUGH INFO': 158})}

# Create sqlite db for titled SciFact wiki pages and titled ID claims

Script: /users/k21190024/study/fact-check-transfer-learning/repos/fever/baseline/src/scripts/build_db_mod.py

In [37]:
conn = sqlite3.connect(scifactdp / "feverised-scifact-titleid.db")
cur = conn.cursor()

cur.execute("select * from documents limit 3")
res = cur.fetchall()

conn.close()

In [38]:
res

[('Microstructural_development_of_human_newborn_cerebral_white_matter_assessed_in_vivo_by_diffusion_tensor_magnetic_resonance_imaging.',
  '4983',
  'Alterations of the architecture of cerebral white matter in the developing human brain can affect cortical development and result in functional disabilities. A line scan diffusion-weighted magnetic resonance imaging (MRI) sequence with diffusion tensor analysis was applied to measure the apparent diffusion coefficient, to calculate relative anisotropy, and to delineate three-dimensional fiber architecture in cerebral white matter in preterm (n = 17) and full-term infants (n = 7). To assess effects of prematurity on cerebral white matter development, early gestation preterm infants (n = 10) were studied a second time at term. In the central white matter the mean apparent diffusion coefficient at 28 wk was high, 1.8 microm2/ms, and decreased toward term to 1.2 microm2/ms. In the posterior limb of the internal capsule, the mean apparent diff

In [5]:
sf_claims = read_data(scifactdp / "scifact_all.jsonl")
sf_claims[129], sf_claims[0]

({'id': 222,
  'claim': 'Ca2+ cycling is a UCP1-independent thermogenic mechanism.',
  'label': 'SUPPORTS',
  'verifiable': 'VERIFIABLE',
  'elab': ['SUPPORTS', 'SUPPORTS'],
  'evidence': [[[None, None, '19205437', 1], [None, None, '19205437', 3]]]},
 {'id': 0,
  'claim': '0-dimensional biomaterials lack inductive properties.',
  'label': 'NOT ENOUGH INFO',
  'verifiable': 'NOT VERIFIABLE',
  'elab': None,
  'evidence': [[[None, None, None, None]]]})

In [4]:
def replace_id_with_titleid(doc):
    if doc["label"] == "NOT ENOUGH INFO":
        return doc
    conn = sqlite3.connect(scifactdp / "feverised-scifact-titleid.db")
    cur = conn.cursor()
    
    evidences = doc["evidence"]
    titleid_evidences = []
    for e in evidences[0]:
        cur.execute("SELECT id FROM documents WHERE original_id = ?", (e[2],))
        res = cur.fetchone()[0]
        titleid_evidences.append([e[0], e[1], res, e[3]])
    new_doc = deepcopy(doc)
    new_doc["evidence"] = titleid_evidences
    
    conn.close()
    
    return new_doc

In [9]:
titleid_sf_claims = Parallel(n_jobs=20)(delayed(replace_id_with_titleid)(doc) for doc in sf_claims)
len(titleid_sf_claims), all([new["id"] == old["id"] for old, new in zip(sf_claims, titleid_sf_claims)])

(1109, True)

In [11]:
write_jsonl(scifactdp / "scifact_all_titleid.jsonl", titleid_sf_claims)

PosixPath('/users/k21190024/study/fact-check-transfer-learning/scratch/dumps/feverised-scifact/scifact_all_titleid.jsonl')